In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from glaft.georaster import Raster

In [3]:
import rasterio
# import matplotlib.pyplot as plt
# from rasterio.plot import show as rio_show
# import matplotlib as mpl
import pandas as pd
import geopandas as gpd
import numpy as np

In [4]:
df = pd.read_csv('../results_2022.csv', dtype=str)
df

,Date,Duration (days),Template size (px),Template size (m),Pixel spacing (px),Pixel spacing (m),Prefilter,Subpixel,Software,Vx,...,pt1_vxdiff,pt1_vydiff,pt2_vxdiff,pt2_vydiff,pt0_vxavgdiff,pt0_vyavgdiff,pt1_vxavgdiff,pt1_vyavgdiff,pt2_vxavgdiff,pt2_vyavgdiff
0,Sen2-20180304-20180314,10,48,480,12,120,Gau,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,...,0.005894817186356094,-0.24796626664900728,0.5122103172194441,0.8053853176141387,-0.0038444487305129282,0.08725667293912226,0.013707317186356094,-0.24796626664900728,0.5469325261961897,0.7776075265908843
1,Sen2-20180304-20180314,10,48,480,12,120,NAOF,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,...,0.017465313984259878,-0.2676457698511035,NaN,NaN,-0.21362812356632954,-0.007943643728092364,0.01833336622844628,-0.2641735459731967,1.3050308140173479,3.134664068272516
2,Sen2-20180304-20180314,10,48,480,12,120,None,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,...,-0.15324545062200834,-0.18089713317288342,3.65775754941108,-3.2525455489097377,-0.17253333220312106,-0.11650750704987228,-0.15498156256096174,-0.18784158092869702,3.5466463853180623,-3.5824067130027553
3,Sen2-20180304-20180314,10,48,480,1,10,Gau,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,...,0.0010556991207103517,-0.27696618111655136,0.5698711991537984,0.7138854031465945,-0.009551633941506266,0.021798445006459777,0.0010556991207103517,-0.27696618111655136,0.54903788568868,0.6756909851931033
4,Sen2-20180304-20180314,10,48,480,1,10,NAOF,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,...,0.022108991871400696,-0.2713151082616328,5.653424491904489,-5.4367135239984865,-0.2523872218012819,-0.052411586626994466,0.018636767993493897,-0.2713151082616328,0.7315494919044887,-5.4210885239984865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,Sen2-20180508-20180627,50,64,640,8,80,Gau,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,...,NaN,NaN,NaN,NaN,-0.04530916930480611,0.02437105767680159,NaN,NaN,NaN,NaN
168,Sen2-20180508-20180627,50,32,320,4,40,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,...,NaN,NaN,NaN,NaN,-0.011477912704790816,-0.05133395484103365,NaN,NaN,NaN,NaN
169,Sen2-20180508-20180627,50,32,320,8,80,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,...,NaN,NaN,NaN,NaN,-0.023014667846172177,-0.031017472172892413,NaN,NaN,NaN,NaN
170,Sen2-20180508-20180627,50,64,640,4,40,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,...,NaN,NaN,NaN,NaN,-0.0368165485968755,-0.06210640555592839,NaN,NaN,NaN,NaN


In [5]:
tmp = Raster(df.loc[0, 'Vx'])
a, b = tmp.value_at_coords(621306.41954208, 6738829.50233354, window=3, return_window=True)
vx_grid = rasterio.open(df.loc[0, 'Vx'])
sample_gen_vx = vx_grid.sample([(621306.41954208, 6738829.50233354)])
vx_sampled = np.array([float(record) for record in sample_gen_vx])

In [6]:
# sampled

In [24]:
gps_files = ['/home/jovyan/Projects/PX_comparison/GPS/Kaskawulsh_2018-04-05_to_2018-03-04_GPS', 
             '/home/jovyan/Projects/PX_comparison/GPS/Kaskawulsh_2018-08-18_to_2018-08-02_GPS',
             '/home/jovyan/Projects/PX_comparison/GPS/Kaskawulsh_2018-03-14_to_2018-03-04_GPS',
             '/home/jovyan/Projects/PX_comparison/GPS/Kaskawulsh_2018-06-27_to_2018-05-08_GPS']

# '/home/jovyan/Projects/PX_comparison/GPS/Kaskawulsh_v2_2018-05-08_to_2018-06-27_GPS.csv'

datestrs = ['LS8-20180304-20180405', 'LS8-20180802-20180818', 'Sen2-20180304-20180314', 'Sen2-20180508-20180627']
datenums = [32, 16, 10, 50]

In [25]:
for gps_file, datestr, datenum in zip(gps_files, datestrs, datenums):
    gps = pd.read_csv(gps_file) 
    # Additional treatment for Sen2-20180508-20180627
    # many of the points here should be (nan, nan), (nan, nan) but nans does not work with rio.sample
    if datestr == 'Sen2-20180508-20180627':
        gps.loc[1, 'end_easting'] = 610481.2868266493
        gps.loc[1, 'end_northing'] = 6737102.953712379
        gps.loc[2, 'end_easting'] = 601790.4387747
        gps.loc[2, 'end_northing'] = 6733753.77267354
        gps = gps.loc[0:2]
    gps = gpd.GeoDataFrame(gps, geometry=gpd.points_from_xy(gps['end_easting'], gps['end_northing']), crs='EPSG:32607')
    # This is beginning coordinates
    gps_xy = list(gps[['end_easting', 'end_northing']].to_records(index=False))
    # if datestr == 'Sen2-20180508-20180627':
    #     gps_xy[-2] = (601790.43877479, 6733753.77267354)  # it should be (nan, nan), (nan, nan) but nans does not work with rio.sample
    #     gps_xy = gps_xy[:-1]
    print(datestr, gps_xy)
    
    gps['vx (m/d)']  = (gps['start_easting'] - gps['end_easting']) / datenum
    gps['vy (m/d)']  = (gps['start_northing'] - gps['end_northing']) / datenum
    
    df_s = df.loc[df['Date'] == datestr]
    for idx, row in df_s.iterrows():
        vx_grid = Raster(row.Vx)
        vy_grid = Raster(row.Vy)
        sampled = []
        for x, y in gps_xy:
            vx_avg, vx_3by3 = vx_grid.value_at_coords(x, y, window=3, return_window=True)
            vy_avg, vy_3by3 = vy_grid.value_at_coords(x, y, window=3, return_window=True)
            vx_3by3[vx_3by3 < -9998] = np.nan
            vy_3by3[vy_3by3 < -9998] = np.nan
            vx_3by3[vx_3by3 == 0.0] = np.nan    #Vmap
            vy_3by3[vy_3by3 == 0.0] = np.nan    #Vmap
            vx_nn = vx_3by3[0, 1, 1]     # nearest neighbor value
            vy_nn = vy_3by3[0, 1, 1]
            if np.any(~np.isnan(vx_3by3)):
                vx_avg = np.nanmean(vx_3by3)
            else:
                vx_avg = np.nan
            if np.any(~np.isnan(vy_3by3)):
                vy_avg = np.nanmean(vy_3by3)
            else:
                vy_avg = np.nan
                
            vx_nn -= float(df.loc[idx, 'SAV-peak-x'])
            vx_avg -= float(df.loc[idx, 'SAV-peak-x'])
            vy_nn -= float(df.loc[idx, 'SAV-peak-y'])
            vy_avg -= float(df.loc[idx, 'SAV-peak-y'])
            
            sampled.append([vx_nn, vx_avg, vy_nn, vy_avg])
        
        sampled = np.array(sampled)
        # print(row.Vx, float(df.loc[idx, 'SAV-peak-x']), float(df.loc[idx, 'SAV-peak-y']), sampled)
        
        df.loc[idx, 'pt0_vxdiff'] = sampled[0, 0] - gps.loc[0, 'vx (m/d)']
        df.loc[idx, 'pt0_vxavgdiff'] = sampled[0, 1] - gps.loc[0, 'vx (m/d)']
        df.loc[idx, 'pt0_vydiff'] = sampled[0, 2] - gps.loc[0, 'vy (m/d)']
        df.loc[idx, 'pt0_vyavgdiff'] = sampled[0, 3] - gps.loc[0, 'vy (m/d)']
        df.loc[idx, 'pt1_vxdiff'] = sampled[1, 0] - gps.loc[1, 'vx (m/d)']
        df.loc[idx, 'pt1_vxavgdiff'] = sampled[1, 1] - gps.loc[1, 'vx (m/d)']
        df.loc[idx, 'pt1_vydiff'] = sampled[1, 2] - gps.loc[1, 'vy (m/d)']
        df.loc[idx, 'pt1_vyavgdiff'] = sampled[1, 3] - gps.loc[1, 'vy (m/d)']
        df.loc[idx, 'pt2_vxdiff'] = sampled[2, 0] - gps.loc[2, 'vx (m/d)']
        df.loc[idx, 'pt2_vxavgdiff'] = sampled[2, 1] - gps.loc[2, 'vx (m/d)']
        df.loc[idx, 'pt2_vydiff'] = sampled[2, 2] - gps.loc[2, 'vy (m/d)']
        df.loc[idx, 'pt2_vyavgdiff'] = sampled[2, 3] - gps.loc[2, 'vy (m/d)']
        
        # vx_grid = rasterio.open(row.Vx)
        # vy_grid = rasterio.open(row.Vy)
        # sample_gen_vx = vx_grid.sample(gps_xy)
        # vx_sampled = np.array([float(record) for record in sample_gen_vx])
        # sample_gen_vy = vy_grid.sample(gps_xy)
        # vy_sampled = np.array([float(record) for record in sample_gen_vy])
        # vx_sampled[vx_sampled < -9998] = np.nan
        # vy_sampled[vy_sampled < -9998] = np.nan
        # vx_sampled -= float(df.loc[idx, 'SAV-peak-x'])
        # vy_sampled -= float(df.loc[idx, 'SAV-peak-y'])
        # df.loc[idx, 'pt0_vxdiff'] = vx_sampled[0] - np.abs(gps.loc[0, 'vx (m/d)'])
        # df.loc[idx, 'pt0_vydiff'] = vy_sampled[0] - np.abs(gps.loc[0, 'vy (m/d)'])
        # df.loc[idx, 'pt1_vxdiff'] = vx_sampled[1] - np.abs(gps.loc[1, 'vx (m/d)'])
        # df.loc[idx, 'pt1_vydiff'] = vy_sampled[1] - np.abs(gps.loc[1, 'vy (m/d)'])        
        # df.loc[idx, 'pt2_vxdiff'] = vx_sampled[2] - np.abs(gps.loc[2, 'vx (m/d)'])
        # df.loc[idx, 'pt2_vydiff'] = vy_sampled[2] - np.abs(gps.loc[2, 'vy (m/d)'])        
        # v_sampled = np.sqrt(vx_sampled ** 2 + vy_sampled ** 2)
        # df.loc[idx, 'pt0_velodiff'] = v_sampled[0] - np.abs(gps.loc[0, 'velocity (m/d)'])
        # df.loc[idx, 'pt1_velodiff'] = v_sampled[1] - np.abs(gps.loc[1, 'velocity (m/d)'])
        # df.loc[idx, 'pt2_velodiff'] = v_sampled[2] - np.abs(gps.loc[2, 'velocity (m/d)'])

LS8-20180304-20180405 [(621306.41954208, 6738829.50233354), (610435.5249175, 6737129.57698521), (601733.22946583, 6733710.66504834)]
LS8-20180802-20180818 [(621363.01607688, 6738895.12164604), (610506.52739125, 6737089.56006354), (601790.43877479, 6733753.77267354)]
Sen2-20180304-20180314 [(621306.41954208, 6738829.50233354), (610435.5249175, 6737129.57698521), (601733.22946583, 6733710.66504834)]
Sen2-20180508-20180627 [(621324.96198502, 6738852.60218059), (610481.28682665, 6737102.95371238), (601790.4387747, 6733753.77267354)]


In [26]:
gps

,Unnamed: 0,date1,date2,start_easting,start_northing,end_easting,end_northing,distance_traveled (m),velocity (m/d),geometry,vx (m/d),vy (m/d)
0,0,2018-06-27,2018-05-08,621348.115449,6.738880e+06,621324.961985,6.738853e+06,36.045525,0.720837,POINT (621324.962 6738852.602),0.463069,0.55252
1,1,2018-06-27,2018-05-08,610481.286827,6.737103e+06,610481.286827,6.737103e+06,NaN,NaN,POINT (610481.287 6737102.954),0.000000,0.00000
2,2,2018-06-27,2018-05-08,NaN,NaN,601790.438775,6.733754e+06,NaN,NaN,POINT (601790.439 6733753.773),NaN,NaN


In [7]:
# gps.loc[0, 'start_easting'] + 1
gps
# gps['end_easting'] - gps['start_easting']

,Unnamed: 0,date1,date2,start_easting,start_northing,end_easting,end_northing,distance_traveled (m),velocity (m/d),label,distance_traveled,velocity,geometry,vx (m/d),vy (m/d)
0,0,2018-05-08,2018-06-27,621324.961985,6.738853e+06,621348.115449,6.738880e+06,36.045525,0.720837,lower,NaN,NaN,POINT (621348.115 6738880.228),0.463069,0.55252
1,1,2018-05-08,2018-06-27,NaN,NaN,610481.286827,6.737103e+06,NaN,NaN,middle,NaN,NaN,POINT (610481.287 6737102.954),NaN,NaN
2,2,2018-05-08,2018-06-27,NaN,NaN,NaN,NaN,NaN,NaN,upper,NaN,NaN,POINT EMPTY,NaN,NaN
3,3,2018-05-08,2018-06-27,NaN,NaN,NaN,NaN,NaN,NaN,arm,NaN,NaN,POINT EMPTY,NaN,NaN


In [27]:
df

,Date,Duration (days),Template size (px),Template size (m),Pixel spacing (px),Pixel spacing (m),Prefilter,Subpixel,Software,Vx,...,pt1_vxdiff,pt1_vydiff,pt2_vxdiff,pt2_vydiff,pt0_vxavgdiff,pt0_vyavgdiff,pt1_vxavgdiff,pt1_vyavgdiff,pt2_vxavgdiff,pt2_vyavgdiff
0,Sen2-20180304-20180314,10,48,480,12,120,Gau,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,...,0.005895,0.082225,0.51221,0.805385,-0.003844,0.087257,0.013707,0.082225,0.546933,0.777608
1,Sen2-20180304-20180314,10,48,480,12,120,NAOF,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,...,0.017465,0.062545,NaN,NaN,-0.213628,-0.007944,0.018333,0.066018,1.305031,3.134664
2,Sen2-20180304-20180314,10,48,480,12,120,None,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,...,-0.153245,0.149294,3.657758,-3.252546,-0.172533,-0.116508,-0.154982,0.14235,3.546646,-3.582407
3,Sen2-20180304-20180314,10,48,480,1,10,Gau,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,...,0.001056,0.053225,0.569871,0.713885,-0.009552,0.021798,0.001056,0.053225,0.549038,0.675691
4,Sen2-20180304-20180314,10,48,480,1,10,NAOF,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,...,0.022109,0.058876,5.653424,-5.436714,-0.252387,-0.052412,0.018637,0.058876,0.731549,-5.421089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,Sen2-20180508-20180627,50,64,640,8,80,Gau,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,...,0.644149,-0.205053,NaN,NaN,-0.045309,0.024371,0.646927,-0.198109,NaN,NaN
168,Sen2-20180508-20180627,50,32,320,4,40,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,...,0.521036,-0.221036,NaN,NaN,-0.030922,-0.040223,0.550202,-0.236314,NaN,NaN
169,Sen2-20180508-20180627,50,32,320,8,80,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,...,0.546999,-0.228497,NaN,NaN,-0.023015,-0.031017,0.556721,-0.242386,NaN,NaN
170,Sen2-20180508-20180627,50,64,640,4,40,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,...,0.534586,-0.259586,NaN,NaN,-0.052094,-0.034329,0.540142,-0.26653,NaN,NaN


In [28]:
df.to_csv('../results_2022.csv', index=False)